---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [73]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [98]:
import re
import datetime as dt

def month_text_to_number(month_text):
    m = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
         'may':5,
         'jun':6,
         'jul':7,
         'aug':8,
         'sep':9,
         'oct':10,
         'nov':11,
         'dec':12
        }
    s = month_text.strip()[:3].lower()

    try:
        out = m[s]
        return out
    except:
        raise ValueError('Not a month: ' + month_text)

def try_build_date(raw_text, day_text=None,month_text=None,year_text=None,month_full_text=None):
    day = 1
    month = 1
    try:
        if month_text is not None:
            month = int(month_text)
            
        if month_full_text is not None:
            month = month_text_to_number(month_full_text)
            
        if day_text is not None:
            day = int(day_text) 
            
        year = int(year_text)     
        if year < 100:
            year += 1900
            
        return dt.datetime(year, month, day)
    except Exception as e: 
        #print(e)        
        #print('Unable to build date', raw_text)
        return None
    
def parse_date(raw_text):
    match = re.search(r'(?P<month>\d{1,2})/(?P<day>\d{1,2})/(?P<year>\d{1,4})', raw_text)
    if match:
        date = try_build_date(raw_text, day_text=match.group('day'), month_text=match.group('month'), year_text=match.group('year'))
        if date is not None:
            return date
    
    match = re.search(r'(?P<month>\d{1,2})-(?P<day>\d{1,2})-(?P<year>\d{1,4})', raw_text)
    if match:
        date = try_build_date(raw_text, day_text=match.group('day'), month_text=match.group('month'), year_text=match.group('year'))
        if date is not None:
            return date
    
    match = re.search(r'(?P<day>\d{1,2}) (?P<month_text>[A-Z][a-z]{2,}) (?P<year>\d{1,4})', raw_text)
    if match:
        date = try_build_date(raw_text, day_text=match.group('day'), month_full_text=match.group('month_text'), year_text=match.group('year'))
        if date is not None:
            return date
    
    match = re.search(r'(?P<month_text>[A-Z][a-z]{2,}),? (?P<year>\d{4})', raw_text)
    if match:
        date = try_build_date(raw_text, month_full_text=match.group('month_text'), year_text=match.group('year'))
        if date is not None:
            return date
    
    match = re.search(r'(?P<month_text>[A-Z][a-z]{2,}) (?P<day>\d{1,2})\W*(?P<year>\d{1,4})', raw_text)
    if match:
        date = try_build_date(raw_text, day_text=match.group('day'), month_full_text=match.group('month_text'), year_text=match.group('year'))
        if date is not None:
            return date
    
    match = re.search(r'(?P<month>\d{1,2})/(?P<year>\d{1,4})', raw_text)
    if match:
        date = try_build_date(raw_text, month_text=match.group('month'), year_text=match.group('year'))
        if date is not None:
            return date
        
    match = re.search(r'(?P<year>\d{4})', raw_text)
    if match:
        date = try_build_date(raw_text, year_text=match.group('year'))
        if date is not None:
            return date
    
    return None

def date_sorter():
        
    df_parsed = pd.DataFrame(df, columns=['raw_date'])
    df_parsed['date'] = df_parsed['raw_date'].apply(parse_date)
    
    print('Parsed dates count', df_parsed['date'].count())
    print('Unparsed dates', df_parsed[df_parsed['date'].isnull()]['raw_date'].unique())
    
    df_parsed['original_index'] = df.index
    
    df_parsed.sort_values('date', inplace=True)
    df_parsed.insert(0, 'date_order', range(0, len(df_parsed)))
    df_parsed.set_index(['date_order'], inplace=True)
    
    return pd.Series(df_parsed['original_index'])

date_sorter()

Parsed dates count 500
Unparsed dates []


date_order
0      447
1        9
2       84
3        2
4       53
5       28
6      474
7      153
8       13
9      129
10      98
11     111
12     225
13      31
14     171
15     191
16     486
17     415
18     335
19      36
20     323
21     405
22     422
23     375
24     380
25     345
26      57
27     214
28     481
29     436
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    198
478    286
479    480
480    431
481    279
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: original_index, Length: 500, dtype: int64